<a href="https://colab.research.google.com/github/igordantasgf/PythonFrameworks/blob/main/Python%20e%20BDs%20Relacionais/py_sql_mongo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **ORM : Object Relational Mapper**
RM ou mapeador objeto relaciona, é um tipo de ferramenta muito utilizada hoje em dia, com o propósito de unir o mundo orietado a objetos e o mundo relacional.
#### Objeto $→$ Modelo Relacional

#### **Vantagens:**
- Menos código
- Melhor manutenção
- Utilização de conectores
- Indicado para CRUDs (é um acrônimo para as quatro basic operations: creeate, read, update, delete)

### **Por que usar?**
- Troca de SGBD mais facilitada (Sistema de Gerenciamento de Banco de Dados)
- Modelo MVC (Model-View-Controller)
- Diminuição do DRY (Don't repeat yourself)
- Evita problemas de segurança

### Criação do Banco de Dados

In [1]:
from IPython.display import clear_output
!pip install pymongo
clear_output()

In [2]:
import sqlalchemy
import pymongo

In [25]:
from sqlalchemy.orm.relationships import foreign
#import sqlalchemy as sqlA
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, inspect, select, func

Base = declarative_base()


class User(Base):
  __tablename__ = "user_account"

  # atributos
  id = Column(Integer, primary_key=True) # chave primaria (EDA)
  name = Column(String)
  fullName = Column(String)

  user = relationship("Address", back_populates="user")

  address = relationship(
      "Address", back_populates="user", cascade="all, delete-orphan"
  )

  def __repr__(self):
    return f"User(id={self.id}, name={self.name}, fullName={self.fullName})"



class Address(Base):
  __tablename__ = "address"

  id = Column(Integer, primary_key=True, autoincrement=True)
  email_address = Column(String(40), nullable=False)
  user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)

  user = relationship("User", back_populates="address")

  def __repr__(self):
    return f"Address (id={self.id}, email={self.email_address})"



print(User.__tablename__)
print(Address.__tablename__)


#Conexão com banco de dados
engine = create_engine("sqlite://")

#Criando as classes como tabelas no banco de dados
Base.metadata.create_all(engine)

insp = inspect(engine)

print(insp.get_table_names())
print(insp.default_schema_name)


user_account
address
['address', 'user_account']
main


In [14]:
from sqlalchemy.orm.session import Session

with Session(engine) as session:
  igor = User(
      name="Igor",
      fullName="Igor Dantas",
      address=[Address(email_address='igordantas@gmail.com')]
  )
  bel = User(
      name="Isabel",
      fullName="Maria Isabel",
      address=[Address(email_address='mariaisabel@gmail.com'),
               Address(email_address='maria_reserva@gmail.com')]
  )
  mari = User(
      name="Mariana",
      fullName="Mariana Dantas"
  )

  # enviando para o BD (persistência de dados)
  session.add_all([igor,bel,mari])

  session.commit()

clear_output()

In [15]:
# Consultas ao banco de dados

stmt = select(User).where(User.name.in_(['Igor','Mariana']))
for user in session.scalars(stmt):
  print(user)

print("\n")

stmt_address = select(Address).where(Address.user_id.in_([2]))
for address in session.scalars(stmt_address):
  print(address)

User(id=1, name=Igor, fullName=Igor Dantas)
User(id=3, name=Mariana, fullName=Mariana Dantas)


Address (id=2, email=mariaisabel@gmail.com)
Address (id=3, email=maria_reserva@gmail.com)


### Funções SQL
- ORDER BY,
- JOIN
- COUNT

In [20]:
# ORDER BY

ordem = select(User).order_by(User.id.desc())
for fn in session.scalars(ordem):
  print(fn)

User(id=3, name=Mariana, fullName=Mariana Dantas)
User(id=2, name=Isabel, fullName=Maria Isabel)
User(id=1, name=Igor, fullName=Igor Dantas)


In [24]:
# JOIN

join = select(User.fullName, Address.email_address).join_from(Address, User)
connection = engine.connect()
results = connection.execute(join).fetchall()
for result in results:
  print(result)

('Igor Dantas', 'igordantas@gmail.com')
('Maria Isabel', 'mariaisabel@gmail.com')
('Maria Isabel', 'maria_reserva@gmail.com')


In [28]:
# COUNT

counts = select(func.count('*')).select_from(User)
for cont in session.scalars(counts):
  print(cont)

3
